In [5]:
pip install pandas-ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=46090249a6b05eb2e4d029f925feec0a7f808d5f116014016ea2d39ef75b0972
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [6]:
import yfinance as yf
import numpy as np
import pandas_ta as ta

In [13]:
# Fetch historical data for a specific stock
stock_data = yf.download('AAPL', start='2020-01-01', end='2023-10-01')

[*********************100%***********************]  1 of 1 completed


In [8]:
display(stock_data.head())

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2020-01-02 00:00:00+00:00,72.796028,75.087502,75.150002,73.797501,74.059998,135480400
2020-01-03 00:00:00+00:00,72.088295,74.357498,75.144997,74.125000,74.287498,146322800
2020-01-06 00:00:00+00:00,72.662720,74.949997,74.989998,73.187500,73.447502,118387200
2020-01-07 00:00:00+00:00,72.320969,74.597504,75.224998,74.370003,74.959999,108872000
2020-01-08 00:00:00+00:00,73.484352,75.797501,76.110001,74.290001,74.290001,132079200


In [14]:
# Calculate daily returns
stock_data['Daily Return'] = stock_data['Adj Close'].pct_change()

# Calculate volatility (standard deviation of daily returns)
volatility = stock_data['Daily Return'].std()
print(f"Volatility: {volatility}")

Volatility: 0.02169533338634344


In [15]:
display(stock_data.head())

Price,Adj Close,Close,High,Low,Open,Volume,Daily Return
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,
Date,,,,,,,
2020-01-02 00:00:00+00:00,72.796028,75.087502,75.150002,73.797501,74.059998,135480400,NaN
2020-01-03 00:00:00+00:00,72.088295,74.357498,75.144997,74.125000,74.287498,146322800,-0.009722
2020-01-06 00:00:00+00:00,72.662720,74.949997,74.989998,73.187500,73.447502,118387200,0.007968
2020-01-07 00:00:00+00:00,72.320969,74.597504,75.224998,74.370003,74.959999,108872000,-0.004703
2020-01-08 00:00:00+00:00,73.484352,75.797501,76.110001,74.290001,74.290001,132079200,0.016086


In [16]:
# Calculate beta (using market index like S&P 500)
sp500 = yf.download('^GSPC', start='2020-01-01', end='2023-10-01')
sp500['Daily Return'] = sp500['Adj Close'].pct_change()

covariance = stock_data['Daily Return'].cov(sp500['Daily Return'])
market_variance = sp500['Daily Return'].var()
beta = covariance / market_variance
print(f"Beta: {beta}")

[*********************100%***********************]  1 of 1 completed

Beta: 1.19583107287933


In [17]:
# Calculate a simple moving average
stock_data['SMA'] = stock_data['Adj Close'].rolling(window=20).mean()

# Calculate Relative Strength Index (RSI)
stock_data['RSI'] = ta.rsi(stock_data['Adj Close'], length=14)

print(stock_data[['SMA', 'RSI']].tail())

investor_profiles = {
    'conservative': {'risk_tolerance': 0.1},
    'moderate': {'risk_tolerance': 0.2},
    'aggressive': {'risk_tolerance': 0.3}
}

Price                             SMA   RSI
Ticker                                     
Date                                       
2023-09-25 00:00:00+00:00  178.667664  None
2023-09-26 00:00:00+00:00  178.258710  None
2023-09-27 00:00:00+00:00  177.578443  None
2023-09-28 00:00:00+00:00  176.735688  None
2023-09-29 00:00:00+00:00  175.907839  None


In [18]:
def recommend_strategy(volatility, beta, profile):
    if volatility < profile['risk_tolerance'] and beta < 1:
        return "Buy"
    elif volatility > profile['risk_tolerance'] and beta < 1:
        return "Hold"
    else:
        return "Sell"

In [19]:
profile = investor_profiles['moderate']
strategy = recommend_strategy(volatility, beta, profile)
print(f"Recommended Strategy: {strategy}")

Recommended Strategy: Sell
